In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path
import base64
from email.message import EmailMessage
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Gmail API 권한 범위 (메일 보내기)
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def main():
    creds = None
    # 기존 토큰이 있으면 불러오기
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # 토큰이 없거나 만료됐으면 새로 인증
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=19999)
        # 토큰 저장
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('gmail', 'v1', credentials=creds)
        # 이메일 메시지 작성
        message = EmailMessage()
        message.set_content('오늘의 구내식당 메뉴입니다. 맛점 하세요~!')
        message['To'] = 'leejh@bccard.com'
        message['From'] = 'leejh900811@gmail.com'
        message['Subject'] = 'Gmail API 테스트'

        # result1.md 파일 읽기
        with open('result1.md', 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 메일 내용 설정
        message.set_content(content)
        
        # base64로 인코딩
        encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        create_message = {'raw': encoded_message}

        # 메일 발송
        send_message = service.users().messages().send(userId="me", body=create_message).execute()
        print(f"Message Id: {send_message['id']}")
    except HttpError as error:
        print(f"An error occurred: {error}")

if __name__ == '__main__':
    main()

Message Id: 196cd043b0a20e35
